In [1]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Dropout,concatenate, UpSampling2D

from tensorflow.keras import Input, Model
from tensorflow.python.keras import regularizers

In [2]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same",kernel_regularizer=regularizers.l2(0.0001), bias_regularizer=regularizers.l2(0.0001))(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same",kernel_regularizer=regularizers.l2(0.0001), bias_regularizer=regularizers.l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [3]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2), strides=2)(x)
    p = Dropout(0.25)(p)
    return x, p

In [4]:
def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [5]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 256)
    s2, p2 = encoder_block(p1, 512)
    
    b1 = conv_block(p2, 1024)
    
    d1 = decoder_block(b1, s2, 512)
    d2 = decoder_block(d1, s1, 256)
    
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model
